# Домашнее задание «Функции и работа с данными»

### Задание 1.

In [2]:
import pandas as pd

In [34]:
"""
Написать функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг

Результат классификации записать в столбец class.
"""
ratings = pd.read_csv('ratings.csv')
movies = pd.read_csv('movies.csv')

joined = ratings.merge(movies, how='left') # объединяем в общую таблицу.

# формируем таблицу нужного вида: индекс - название фильма - средний рейтинг.  
movies_rating = joined.groupby('title').mean().reset_index()[['title', 'rating']] 

In [35]:
def class_(row):
    """Определяет класс фильма по его рейтингу.
    """
    if row['rating'] <= 2.0:
        return 'низкий рейтинг'
    elif row['rating'] >= 2.0 and row['rating'] <= 4.0:
        return'средний рейтинг'
    elif row['rating'] > 4.0:
        return 'высокий рейтинг'

In [37]:
movies_rating['class'] = movies_rating.apply(class_, axis=1)
movies_rating.head(10)

,title,rating,class
0,"""Great Performances"" Cats (1998)",1.750000,низкий рейтинг
1,$9.99 (2008),3.833333,средний рейтинг
2,'Hellboy': The Seeds of Creation (2004),2.000000,низкий рейтинг
3,'Neath the Arizona Skies (1934),0.500000,низкий рейтинг
4,'Round Midnight (1986),2.250000,средний рейтинг
5,'Salem's Lot (2004),3.500000,средний рейтинг
6,'Til There Was You (1997),2.625000,средний рейтинг
7,"'burbs, The (1989)",3.052632,средний рейтинг
8,'night Mother (1986),5.000000,высокий рейтинг
9,(500) Days of Summer (2009),3.755556,средний рейтинг


### Задание 2.

In [6]:
"""Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую
принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то
в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города,
то ставим ‘undefined’

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {

1 'Центр': ['москва', 'тула', 'ярославль'],
2 
3 'Северо-Запад': ['петербург', 'псков', 'мурманск'],
4
5 'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']

}

Результат классификации запишите в отдельный столбец region.
"""
geo_data = {
           'Центр': ['москва', 'тула', 'ярославль'],
           'Северо-Запад': ['петербург', 'псков', 'мурманск'],
           'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
           }
# делаем отдельный список всех городов из значений geo_data.
cities = []
for values in geo_data.values():
    for city in values:
        cities.append(city)  

In [7]:
dataframe = pd.read_csv('keywords.csv')

In [63]:
def region(row):
    """Выполняет поисковый запрос по названию города.
    """
    for city_name in row['keyword'].split(' '):
        if city_name in cities:
            for items in geo_data.items():
                if city_name in items[1]:
                        return items[0]
    return 'undefined'

In [64]:
dataframe['region'] = dataframe.apply(region, axis=1)

In [65]:
dataframe.head()

,keyword,shows,region
0,вк,64292779,undefined
1,одноклассники,63810309,undefined
2,порно,41747114,undefined
3,ютуб,39995567,undefined
4,вконтакте,21014195,undefined


In [68]:
dataframe[(dataframe['region'] != 'undefined')]

,keyword,shows,region
127,авито москва,979292,Центр
370,авито ру санкт петербург,425134,Северо-Запад
849,авито ярославль,209581,Центр
1063,фарпост владивосток,176951,Дальний Восток
1236,банк санкт петербург,174375,Северо-Запад
...,...,...,...
99292,афиша кино москва на сегодня,3704,Центр
99590,авито ярославль автомобили с пробегом,3673,Центр
99808,полармед мурманск запись на прием,3719,Северо-Запад
99890,яндекс метро москва,4548,Центр


In [69]:
dataframe['region'].value_counts()

undefined         99406
Центр               384
Северо-Запад        146
Дальний Восток       64
Name: region, dtype: int64

### Задание 3.

In [47]:
"""Проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.
При этом не нужно затрагивать субьективные факторы выставления этих рейтингов, а пройтись по следующему
алгоритму:

1) В переменную years записать список из всех годов с 1950 по 2010.
2) Написать функцию production_year, которая каждой строке из названия фильма выставляет год выпуска.
Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому использоывть следующий алгоритм:

  1. Для каждой строки пройтись по всем годам списка years.
  2. Если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска.
  3. Если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год.
  
3) Записать год выпуска фильма по алгоритму пункта 2 в новый столбец 'year'.
4) Посчитать средний рейтинг всех фильмов для каждого значения столбца 'year' и отсортировать результат
по убыванию рейтинга.
""" 
ratings = pd.read_csv('ratings.csv')
movies = pd.read_csv('movies.csv')

joined = ratings.merge(movies, how='left') # объединяем в общую таблицу.

# формируем таблицу нужного вида: индекс - название фильма - средний рейтинг.  
movies_rating = joined.groupby('title').mean().reset_index()[['title', 'rating']]

In [51]:
years = [str(year + 1950) for year in range(61)] # создаем список из всех годов с 1950 по 2010.

In [54]:
def production_year(row):
    """ Для каждой строки из названия фильма выставляет год выпуска.
    """
    for year in years:
        if year in row['title']:
            return year
    return '1900'

In [76]:
movies_rating['year'] = movies_rating.apply(production_year, axis=1)
movies_rating.sort_values('rating', ascending=False).head()

,title,rating,year
4161,Ivan Vasilievich: Back to the Future (Ivan Vas...,5.0,1973
299,Alien Escape (1995),5.0,1995
1125,Boiling Point (1993),5.0,1993
1131,Bone Tomahawk (2015),5.0,1900
1145,Borgman (2013),5.0,1900
